### **Fake News Classifier Using LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [129]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [130]:
import pandas as pd

In [131]:
df=pd.read_csv('/gdrive/My Drive/LSTM/train.csv')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [132]:
# Drop Nan Values

df=df.dropna()

In [133]:
# Get the Independent Features

X=df.drop('label',axis=1)

In [134]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [135]:
## Get the Dependent features

y=df['label']

In [136]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [137]:
X.shape

(18285, 4)

In [138]:
y.shape

(18285,)

In [139]:
import tensorflow as tf

In [140]:
tf.__version__

'2.4.0'

In [141]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [142]:
# Vocabulary size

voc_size=5000

#### **Onehot Representation**

In [143]:
messages=X.copy()

In [144]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [145]:
messages.reset_index(inplace=True)

In [146]:
import nltk
import re
from nltk.corpus import stopwords

In [147]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [148]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [149]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [150]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0:5]

[[279, 2984, 2660, 462, 4473, 3428, 4418, 4177, 1979, 1009],
 [177, 3367, 1057, 1746, 712, 4412, 307],
 [2760, 831, 816, 4844],
 [3899, 3669, 2044, 4536, 3125, 2674],
 [4458, 712, 2263, 3639, 4937, 3364, 712, 1363, 1815, 4613]]

### **Embedding Representation**

In [151]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4177 1979 1009]
 [   0    0    0 ...  712 4412  307]
 [   0    0    0 ...  831  816 4844]
 ...
 [   0    0    0 ... 4869 4771 1088]
 [   0    0    0 ... 1607 2843 1087]
 [   0    0    0 ... 1386 2150 1344]]


In [152]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  279,
       2984, 2660,  462, 4473, 3428, 4418, 4177, 1979, 1009], dtype=int32)

In [153]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [154]:
len(embedded_docs),y.shape

(18285, (18285,))

In [155]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [156]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### **Model Training**

In [158]:
# Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 13ms/step - loss: 0.4803 - accuracy: 0.7412 - val_loss: 0.1942 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1378 - accuracy: 0.9452 - val_loss: 0.2020 - val_accuracy: 0.9210
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0871 - accuracy: 0.9704 - val_loss: 0.2214 - val_accuracy: 0.9215
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0619 - accuracy: 0.9793 - val_loss: 0.2620 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0374 - accuracy: 0.9876 - val_loss: 0.3387 - val_accuracy: 0.9094
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0219 - accuracy: 0.9933 - val_loss: 0.3563 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0193 - accuracy: 0.9941 - val_loss: 0.4095 - val_accuracy: 0.9137
E

### **Adding Dropout**

You can add this code for dropout

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### **Performance Metrics And Accuracy**

In [159]:
#y_pred=model.predict_classes(X_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [160]:
from sklearn.metrics import confusion_matrix

In [161]:
confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 220, 2396]])

In [162]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9111847555923778